In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get update -y
!apt-get install -y xvfb libxi6 libgconf-2-4
# Try apt blender (often 3.x on recent images). If it fails/too old, uncomment the wget block below.
!apt-get install -y blender

# # Fallback (uncomment if apt blender is missing/old):
# !wget -q https://mirror.clarkson.edu/blender/release/Blender3.6/blender-3.6.11-linux-x64.tar.xz -O /content/blender.tar.xz
# !tar -xJf /content/blender.tar.xz -C /content/
# !ln -s /content/blender-3.6.11-linux-x64/blender /usr/local/bin/blender


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,931 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,520 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jamm

In [ ]:
BASE_DIR = "/content/drive/MyDrive/DesignQA"
OUT_DIR  = f"{BASE_DIR}/designqa_synth_fusion"   # dataset output
CKPT_DIR = f"{BASE_DIR}/checkpoints"             # model checkpoints
import os
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(CKPT_DIR, exist_ok=True)
print("Saving dataset to:", OUT_DIR)
print("Saving checkpoints to:", CKPT_DIR)

Saving dataset to: /content/drive/MyDrive/DesignQA/designqa_synth_fusion
Saving checkpoints to: /content/drive/MyDrive/DesignQA/checkpoints


In [ ]:
%%bash
cat > /content/blender_synth_fusion.py << 'PY'
import bpy, math, random, json, argparse, os, sys
from mathutils import Vector

# ---------------- args ----------------
def parse_args():
    argv = sys.argv
    argv = argv[argv.index('--')+1:] if '--' in argv else []
    p = argparse.ArgumentParser()
    p.add_argument('--out', type=str, required=True)
    p.add_argument('--n', type=int, default=8, help='samples per class')
    p.add_argument('--seed', type=int, default=42)
    p.add_argument('--img_size', type=int, default=1024)
    p.add_argument('--orthographic', action='store_true', default=True)
    p.add_argument('--hard_negatives', action='store_true')
    p.add_argument('--rng', type=float, default=0.015, help='pose jitter amplitude')
    p.add_argument('--overlay_viewcube', action='store_true', help='draw small view-cube overlay (needs Pillow)')
    return p.parse_args(argv)

A = parse_args()
random.seed(A.seed)
for sub in ['definition','presence/pos','presence/neg','meta']:
    os.makedirs(os.path.join(A.out, sub), exist_ok=True)

# ---------------- scene: Fusion/SolidWorks vibe ----------------
bpy.ops.wm.read_factory_settings(use_empty=True)
bpy.context.scene.render.engine = 'BLENDER_EEVEE'
bpy.context.scene.render.resolution_x = A.img_size
bpy.context.scene.render.resolution_y = A.img_size
bpy.context.scene.render.film_transparent = False

# world: very light grey ambient
world = bpy.data.worlds.new("World"); world.use_nodes = True
bpy.context.scene.world = world
bg = world.node_tree.nodes["Background"]
bg.inputs[0].default_value = (0.98, 0.98, 0.98, 1.0)  # almost white
bg.inputs[1].default_value = 1.0

# soft area lights for depth (no outlines)
def add_lights():
    def area(E, loc, size=5.0):
        ld=bpy.data.lights.new(name=f"area_{loc}", type='AREA')
        ld.energy=E; ld.size=size
        lo=bpy.data.objects.new(name=f"area_{loc}", object_data=ld); lo.location=loc
        bpy.context.collection.objects.link(lo)
    area(1500,(6,-6,6)); area(900,(-6,2,4)); area(700,(0,8,5))

# materials: flat-ish, slightly specular (CAD)
def mat(name, rgba):
    m = bpy.data.materials.new(name); m.use_nodes = True
    bsdf = m.node_tree.nodes["Principled BSDF"]
    bsdf.inputs['Base Color'].default_value = rgba
    bsdf.inputs['Roughness'].default_value = 0.75
    bsdf.inputs['Specular'].default_value = 0.15
    return m
MAT_GREY  = mat("Grey",(0.72,0.72,0.72,1))
MAT_DARK  = mat("Dark",(0.35,0.35,0.35,1))
MAT_RUB   = mat("Rubber",(0.18,0.18,0.18,1))
MAT_PINK  = mat("Pink",(1.0,0.2,0.7,1))  # Definition highlight

def shade_all_flat():
    for obj in bpy.data.objects:
        if obj.type == 'MESH':
            bpy.context.view_layer.objects.active = obj
            bpy.ops.object.shade_flat()

# ---------------- cameras & views ----------------
def look_at(obj, target):
    d = (target - obj.location).normalized()
    obj.rotation_euler = d.to_track_quat('-Z', 'Y').to_euler()

def add_cam(name):
    cam_data = bpy.data.cameras.new(name)
    cam_obj = bpy.data.objects.new(name, cam_data)
    bpy.context.collection.objects.link(cam_obj)
    cam_data.type = 'ORTHO' if A.orthographic else 'PERSP'
    cam_data.ortho_scale = 7.2 if cam_data.type == 'ORTHO' else 50
    if cam_data.type != 'ORTHO': cam_data.lens = 50
    return cam_obj

VIEWS = {
    "top":    Vector(( 0.0,  0.0,  9.0)),
    "bottom": Vector(( 0.0,  0.0, -9.0)),
    "front":  Vector(( 0.0, -9.0,  1.6)),
    "back":   Vector(( 0.0,  9.0,  1.6)),
    "left":   Vector(( 9.0,  0.0,  1.6)),
    "iso":    Vector(( 7.0, -7.0,  4.5)),
}
def set_cam(cam, loc, target=Vector((0,0,0))):
    cam.location = Vector(loc); look_at(cam, target)

# ---------------- primitive helpers ----------------
def add_cyl(r=0.05, d=1.0, loc=(0,0,0), rot=(0,0,0), m=MAT_GREY):
    bpy.ops.mesh.primitive_cylinder_add(radius=r, depth=d, location=loc,
                                        rotation=tuple(map(math.radians,rot)))
    o=bpy.context.active_object; o.data.materials.clear(); o.data.materials.append(m); return o
def add_box(size=(1,1,1), loc=(0,0,0), m=MAT_GREY):
    bpy.ops.mesh.primitive_cube_add(size=1, location=loc)
    o=bpy.context.active_object; o.scale=(size[0]/2,size[1]/2,size[2]/2)
    o.data.materials.clear(); o.data.materials.append(m); return o
def add_torus(maj=0.4, minr=0.06, loc=(0,0,0), rot=(0,0,0), m=MAT_GREY):
    bpy.ops.mesh.primitive_torus_add(major_radius=maj, minor_radius=minr, location=loc,
                                     rotation=tuple(map(math.radians,rot)))
    o=bpy.context.active_object; o.data.materials.clear(); o.data.materials.append(m); return o

# ---------------- wheel/suspension corner ----------------
def add_wheel_corner(x, y, z=0.35):
    objs=[]
    tire=add_torus(0.35,0.09,(x,y,z),(90,0,0),MAT_RUB)
    hub =add_cyl(0.07,0.25,(x,y,z),(90,0,0),MAT_DARK)
    disc=add_torus(0.12,0.01,(x-0.02,y,z),(90,0,0),MAT_GREY)
    cal =add_box((0.10,0.06,0.08),(x+0.05,y,z),MAT_DARK)
    upr =add_box((0.12,0.12,0.25),(x-0.05,y, z-0.05),MAT_GREY)
    arU =add_cyl(0.015,0.6,(x-0.4,y-0.1,z+0.15),(0,90,25),MAT_GREY)
    arL =add_cyl(0.018,0.6,(x-0.4,y+0.1,z-0.05),(0,90,-25),MAT_GREY)
    pr  =add_cyl(0.012,0.5,(x-0.35,y-0.15,z+0.05),(35,0,15),MAT_DARK)
    rk  =add_box((0.15,0.06,0.06),(x-0.55,y-0.20,z+0.08),MAT_GREY)
    dp  =add_cyl(0.02,0.35,(x-0.7,y-0.25,z+0.05),(90,20,0),MAT_DARK)
    arb =add_cyl(0.01,0.9,(0.0,0.95,0.42),(0,0,0),MAT_GREY)
    objs += [tire,hub,disc,cal,upr,arU,arL,pr,rk,dp,arb]
    return objs

# ---------------- rich vehicle proxy ----------------
def build_vehicle():
    add_lights()
    objs=[]
    # rails & cross members
    objs += [add_cyl(0.045,3.0,( 0.27,0.0,0.35),(0,90,0),MAT_DARK)]
    objs += [add_cyl(0.045,3.0,(-0.27,0.0,0.35),(0,90,0),MAT_DARK)]
    for y in (-1.5,-1.1,-0.7,-0.3,0.1,0.5,0.9,1.3,1.7):
        objs += [add_cyl(0.03,0.55,(0.0,y,0.35),(90,0,0),MAT_DARK)]
    # bulkheads / panels
    objs += [add_box((0.60,0.05,0.60),(0.0,1.10,0.55),MAT_GREY)]
    objs += [add_box((0.60,0.05,0.70),(0.0,-0.75,0.60),MAT_GREY)]
    # side impact tubes
    objs += [add_box((1.8,0.06,0.46),(0.0,0.27,0.55),MAT_DARK)]
    objs += [add_box((1.8,0.06,0.30),(0.0,0.10,0.45),MAT_DARK)]
    # hoops
    objs += [add_torus(0.50,0.03,(0.0,0.5,0.82),(0,90,0),MAT_GREY)]
    objs += [add_torus(0.58,0.035,(0.0,-0.25,0.92),(0,90,0),MAT_GREY)]
    # steering & pedals
    objs += [add_torus(0.18,0.02,(0.0,-0.22,0.94),(0,0,0),MAT_DARK)]
    objs += [add_cyl(0.015,0.75,(0.0,0.20,0.78),(35,0,0),MAT_DARK)]
    objs += [add_box((0.65,0.08,0.08),(0.0,0.96,0.32),MAT_GREY)]
    objs += [add_box((0.25,0.25,0.15),(0.0,0.85,0.36),MAT_GREY)]
    objs += [add_cyl(0.02,0.18,(-0.1,0.75,0.45),(0,90,0),MAT_GREY)]
    # seat/headrest/harness
    objs += [add_box((0.50,0.60,0.92),(0.0,-0.10,0.56),MAT_GREY)]
    objs += [add_box((0.30,0.06,0.25),(0.0,-0.50,1.00),MAT_GREY)]
    objs += [add_cyl(0.005,0.62,(0.10,-0.20,0.90),(90,0,20),MAT_DARK)]
    # cooling + aero
    objs += [add_box((0.55,0.08,0.55),(0.65,0.30,0.56),MAT_GREY)]   # radiator
    objs += [add_box((0.73,0.26,0.46),(0.70,0.30,0.46),MAT_GREY)]   # sidepod
    objs += [add_box((1.00,0.08,0.05),(0.0,1.95,0.84),MAT_GREY)]    # wing
    objs += [add_box((1.00,0.70,0.07),(0.0,-1.55,0.22),MAT_GREY)]   # diffuser
    # accumulator + jacking bar + floor
    objs += [add_box((0.60,0.35,0.30),(-0.2,-1.10,0.45),MAT_GREY)]
    objs += [add_cyl(0.03,0.70,(0.0,-1.70,0.25),(90,0,0),MAT_GREY)]
    objs += [add_box((2.0,2.0,0.04),(0.0,0.0,0.20),MAT_GREY)]
    # wheel corners
    objs += add_wheel_corner( 1.00, 1.25)
    objs += add_wheel_corner(-1.00, 1.25)
    objs += add_wheel_corner( 1.00,-1.25)
    objs += add_wheel_corner(-1.00,-1.25)
    return objs

# ---------------- 31 component proxies ----------------
def comp_accumulator_container():  return [add_box((0.60,0.35,0.30),(-0.2,-1.10,0.45),MAT_GREY)]
def comp_impact_attenuator():      return [add_box((0.35,0.50,0.25),(0.0,1.60,0.46),MAT_GREY)]
def comp_front_hoop():             return [add_torus(0.50,0.03,(0.0,0.5,0.82),(0,90,0),MAT_GREY)]
def comp_main_hoop():              return [add_torus(0.58,0.035,(0.0,-0.25,0.92),(0,90,0),MAT_GREY)]
def comp_side_impact_structure():  return [add_box((1.8,0.06,0.46),(0.0,0.27,0.55),MAT_DARK)]
def comp_front_bulkhead():         return [add_box((0.60,0.05,0.60),(0.0,1.10,0.55),MAT_GREY)]
def comp_rear_bulkhead():          return [add_box((0.60,0.05,0.70),(0.0,-0.75,0.60),MAT_GREY)]
def comp_steering_wheel():         return [add_torus(0.18,0.02,(0.0,-0.22,0.94),(0,0,0),MAT_DARK)]
def comp_steering_column():        return [add_cyl(0.015,0.75,(0.0,0.20,0.78),(35,0,0),MAT_DARK)]
def comp_steering_rack():          return [add_box((0.65,0.08,0.08),(0.0,0.96,0.32),MAT_GREY)]
def comp_upper_control_arm():      return [add_cyl(0.015,0.6,(0.55,1.10,0.50),(0,90,25),MAT_GREY)]
def comp_upright():                return [add_box((0.12,0.12,0.25),(0.95,1.25,0.30),MAT_GREY)]
def comp_pushrod():                return [add_cyl(0.012,0.5,(0.65,1.0,0.55),(35,0,15),MAT_DARK)]
def comp_rocker():                 return [add_box((0.15,0.06,0.06),(0.45,0.95,0.55),MAT_GREY)]
def comp_damper():                 return [add_cyl(0.02,0.35,(0.7,0.9,0.45),(90,20,0),MAT_DARK)]
def comp_antiroll_bar():           return [add_cyl(0.01,0.9,(0.0,0.95,0.42),(0,0,0),MAT_GREY)]
def comp_brake_caliper():          return [add_box((0.10,0.06,0.08),(1.05,1.25,0.35),MAT_DARK)]
def comp_brake_disc():             return [add_torus(0.12,0.01,(0.98,1.25,0.35),(90,0,0),MAT_GREY)]
def comp_brake_master_cylinder():  return [add_cyl(0.02,0.18,(-0.1,0.75,0.45),(0,90,0),MAT_GREY)]
def comp_pedal_box():              return [add_box((0.25,0.25,0.15),(0.0,0.85,0.36),MAT_GREY)]
def comp_seat():                   return [add_box((0.50,0.60,0.92),(0.0,-0.10,0.56),MAT_GREY)]
def comp_headrest():               return [add_box((0.30,0.06,0.25),(0.0,-0.50,1.00),MAT_GREY)]
def comp_harness():                return [add_cyl(0.005,0.62,(0.10,-0.20,0.90),(90,0,20),MAT_DARK)]
def comp_firewall():               return [add_box((0.60,0.05,0.70),(0.0,-0.75,0.60),MAT_GREY)]
def comp_radiator():               return [add_box((0.55,0.08,0.55),(0.65,0.30,0.56),MAT_GREY)]
def comp_sidepod():                return [add_box((0.73,0.26,0.46),(0.70,0.30,0.46),MAT_GREY)]
def comp_diffuser():               return [add_box((1.00,0.70,0.07),(0.0,-1.55,0.22),MAT_GREY)]
def comp_wing():                   return [add_box((1.00,0.08,0.05),(0.0,1.95,0.84),MAT_GREY)]
def comp_chassis_frame():          return [add_cyl(0.045,3.0,(0.27,0.0,0.35),(0,90,0),MAT_DARK)]
def comp_floor_undertray():        return [add_box((2.0,2.0,0.04),(0.0,0.0,0.20),MAT_GREY)]
def comp_jacking_point():          return [add_cyl(0.03,0.70,(0.0,-1.70,0.25),(90,0,0),MAT_GREY)]

COMPONENTS = {
    "accumulator_container": comp_accumulator_container,
    "impact_attenuator": comp_impact_attenuator,
    "front_hoop": comp_front_hoop,
    "main_hoop": comp_main_hoop,
    "side_impact_structure": comp_side_impact_structure,
    "front_bulkhead": comp_front_bulkhead,
    "rear_bulkhead": comp_rear_bulkhead,
    "steering_wheel": comp_steering_wheel,
    "steering_column": comp_steering_column,
    "steering_rack": comp_steering_rack,
    "upper_control_arm": comp_upper_control_arm,
    "upright": comp_upright,
    "pushrod": comp_pushrod,
    "rocker": comp_rocker,
    "damper": comp_damper,
    "antiroll_bar": comp_antiroll_bar,
    "brake_caliper": comp_brake_caliper,
    "brake_disc": comp_brake_disc,
    "brake_master_cylinder": comp_brake_master_cylinder,
    "pedal_box": comp_pedal_box,
    "seat": comp_seat,
    "headrest": comp_headrest,
    "harness": comp_harness,
    "firewall": comp_firewall,
    "radiator": comp_radiator,
    "sidepod": comp_sidepod,
    "diffuser": comp_diffuser,
    "wing": comp_wing,
    "chassis_frame": comp_chassis_frame,
    "floor_undertray": comp_floor_undertray,
    "jacking_point": comp_jacking_point,
}

# ---------------- utils ----------------
def clear_scene():
    bpy.ops.object.select_all(action='SELECT'); bpy.ops.object.delete(use_global=False)
    for b in list(bpy.data.meshes):
        if b.users == 0: bpy.data.meshes.remove(b)

def bbox_center(objs):
    mins=Vector((1e9,1e9,1e9)); maxs=Vector((-1e9,-1e9,-1e9))
    for o in objs:
        for v in o.bound_box:
            w = o.matrix_world @ Vector(v)
            mins = Vector((min(mins.x,w.x),min(mins.y,w.y),min(mins.z,w.z)))
            maxs = Vector((max(maxs.x,w.x),max(maxs.y,w.y),max(maxs.z,w.z)))
    return (mins+maxs)*0.5

def render_to(path, cam):
    bpy.context.scene.camera = cam
    bpy.context.scene.render.filepath = path
    bpy.ops.render.render(write_still=True)

def save_json(path, data):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path,'w') as f: json.dump(data, f, indent=2)

# ------- optional view-cube overlay (Pillow) -------
VIEW_LABELS = {
    "top":"TOP","bottom":"BOTTOM","front":"FRONT","back":"BACK","left":"LEFT","iso":"ISO"
}
def draw_viewcube(image_path, label):
    if not A.overlay_viewcube:
        return
    try:
        from PIL import Image, ImageDraw, ImageFont
    except Exception:
        return  # Pillow not available inside Blender python
    im = Image.open(image_path).convert("RGB")
    w,h = im.size
    sz = max(60, w//10)        # cube size
    pad = max(12, w//80)       # padding from border
    x0 = w - sz - pad
    y0 = pad
    x1 = w - pad
    y1 = pad + sz
    draw = ImageDraw.Draw(im)
    # cube face
    draw.rectangle([x0,y0,x1,y1], outline=(80,80,80), width=3, fill=(245,245,245))
    # little 3D edges
    draw.line([x0,y0, x0+sz//4,y0+sz//4], fill=(80,80,80), width=3)
    draw.line([x1,y0, x1-sz//4,y0+sz//4], fill=(80,80,80), width=3)
    # text
    txt = VIEW_LABELS.get(label,label.upper())
    try:
        font = ImageFont.truetype("DejaVuSans.ttf", size=max(16, sz//4))
    except Exception:
        font = ImageFont.load_default()
    tw,th = draw.textsize(txt, font=font)
    draw.text((x0+(sz-tw)//2, y0+(sz-th)//2), txt, fill=(60,60,60), font=font)
    im.save(image_path)

# ---------------- outputs ----------------
def sixpanel_definition(key, sid):
    out_dir = os.path.join(A.out, "definition", key); os.makedirs(out_dir, exist_ok=True)
    tmp = os.path.join(out_dir, f"tmp_{sid}"); os.makedirs(tmp, exist_ok=True)

    cams={}
    order=list(VIEWS.keys())
    for name,loc in VIEWS.items():
        cams[name]=add_cam(f"cam_{name}")
        set_cam(cams[name], loc, Vector((0,0,0)))

    # render each view, overlay cube, stitch
    paths=[]
    for name in order:
        p=os.path.join(tmp,f"{name}.png")
        render_to(p,cams[name])
        draw_viewcube(p, name)
        paths.append(p)

    out_img=os.path.join(out_dir, f"{sid}_sixpanel.png")
    try:
        from PIL import Image
        imgs=[Image.open(p).convert("RGB") for p in paths]; w,h=imgs[0].size
        grid=Image.new('RGB',(w*3,h*2),(255,255,255))
        for i,im in enumerate(imgs):
            grid.paste(im, ((i%3)*w,(i//3)*h))
        grid.save(out_img)
        for p in paths: os.remove(p)
        os.rmdir(tmp)
    except Exception:
        # If Pillow missing, just keep singles in tmp folder (still usable)
        out_img = paths[-1]
    return order, out_img

def presence_pair(key, sid, targets, match_view):
    cam=add_cam("cam_close")
    direction=VIEWS[match_view].normalized()
    center=bbox_center(targets)
    cam.location = center + direction * (-2.2)
    look_at(cam, center)

    pos=os.path.join(A.out,"presence/pos",key,f"{sid}_closeup.png")
    os.makedirs(os.path.dirname(pos), exist_ok=True)
    render_to(pos, cam)
    draw_viewcube(pos, match_view)

    if A.hard_negatives:
        # Aim at a visually similar distractor (class-specific), otherwise a chassis tube
        distractors = {
            "pushrod": "upper_control_arm",
            "upper_control_arm": "pushrod",
            "rocker": "damper",
            "damper": "rocker",
            "brake_caliper": "brake_disc",
            "brake_disc": "brake_caliper",
            "front_hoop": "main_hoop",
            "main_hoop": "front_hoop",
            "steering_wheel": "steering_column",
            "steering_column": "steering_wheel",
        }
        alt_key = distractors.get(key, "chassis_frame")
        alt = COMPONENTS[alt_key]()
        alt_center = bbox_center(alt)
        cam.location = alt_center + direction * (-2.2); look_at(cam, alt_center)
        neg=os.path.join(A.out,"presence/neg",key,f"{sid}_closeup.png")
        os.makedirs(os.path.dirname(neg), exist_ok=True)
        render_to(neg, cam)
        draw_viewcube(neg, match_view)
        # delete alt objs
        bpy.ops.object.select_all(action='DESELECT')
        for o in alt: o.select_set(True)
        bpy.ops.object.delete(use_global=False)
    else:
        # simple negative = hide the target
        for o in targets: o.hide_render=True; o.hide_set(True)
        neg=os.path.join(A.out,"presence/neg",key,f"{sid}_closeup.png")
        os.makedirs(os.path.dirname(neg), exist_ok=True)
        render_to(neg, cam)
        draw_viewcube(neg, match_view)
        for o in targets: o.hide_render=False; o.hide_set(False)

    return pos, neg

# ---------------- main ----------------
def build_scene(target_key):
    clear_scene()
    base = build_vehicle()
    tgt  = COMPONENTS[target_key]()
    shade_all_flat()
    # tiny randomization
    world.node_tree.nodes["Background"].inputs[1].default_value = random.uniform(0.95, 1.05)
    for o in base + tgt:
        o.location.x += random.uniform(-A.rng, A.rng)
        o.location.y += random.uniform(-A.rng, A.rng)
        o.location.z += random.uniform(-A.rng*0.7, A.rng*0.7)
    return base, tgt

def highlight(objs, on=True):
    for o in objs: o.active_material = MAT_PINK if on else MAT_GREY

def main():
    keys=list(COMPONENTS.keys())
    for key in keys:
        for i in range(A.n):
            sid=f"{key}_{i:05d}"
            base, tgt = build_scene(key)
            # Definition (highlight on)
            highlight(tgt, True)
            views, siximg = sixpanel_definition(key, sid)
            # Presence (matched orientation, highlight off)
            mv=random.choice(views)
            highlight(tgt, False)
            pos,neg = presence_pair(key, sid, tgt, mv)
            # meta
            meta={
                "component": key,
                "id": sid,
                "six_views_order": views,
                "presence_match_view": mv,
                "paths": {"definition_sixpanel": siximg, "presence_pos": pos, "presence_neg": neg},
                "style": {"fusion_like": True, "ortho": True, "flat_shading": True, "viewcube": bool(A.overlay_viewcube)},
                "hard_negatives": bool(A.hard_negatives)
            }
            save_json(os.path.join(A.out,"meta",key,f"{sid}.json"), meta)
            # cleanup
            bpy.ops.object.select_all(action='SELECT'); bpy.ops.object.delete(use_global=False)

if __name__=="__main__":
    main()
PY


In [ ]:

# now render to Drive
!xvfb-run -s "-screen 0 1280x1024x24" blender -b -P /content/blender_synth_fusion.py -- \
  --out "$OUT_DIR" --n 6 --seed 7 --img_size 1024 --hard_negatives --overlay_viewcube


Streaming output truncated to the last 5000 lines.
Fra:1 Mem:41.32M (Peak 41.33M) | Time:00:00.05 | Syncing Cylinder.020
Fra:1 Mem:41.34M (Peak 41.34M) | Time:00:00.05 | Syncing Torus.005
Fra:1 Mem:41.41M (Peak 41.42M) | Time:00:00.05 | Syncing Cylinder.021
Fra:1 Mem:41.42M (Peak 41.43M) | Time:00:00.05 | Syncing Torus.006
Fra:1 Mem:41.50M (Peak 41.51M) | Time:00:00.05 | Syncing Cube.017
Fra:1 Mem:41.51M (Peak 41.51M) | Time:00:00.06 | Syncing Cube.018
Fra:1 Mem:41.51M (Peak 41.51M) | Time:00:00.06 | Syncing Cylinder.022
Fra:1 Mem:41.52M (Peak 41.53M) | Time:00:00.06 | Syncing Cylinder.023
Fra:1 Mem:41.54M (Peak 41.54M) | Time:00:00.06 | Syncing Cylinder.024
Fra:1 Mem:41.55M (Peak 41.55M) | Time:00:00.06 | Syncing Cube.019
Fra:1 Mem:41.55M (Peak 41.55M) | Time:00:00.06 | Syncing Cylinder.025
Fra:1 Mem:41.56M (Peak 41.57M) | Time:00:00.06 | Syncing Cylinder.026
Fra:1 Mem:41.57M (Peak 41.58M) | Time:00:00.06 | Syncing Torus.007
Fra:1 Mem:41.65M (Peak 41.65M) | Time:00:00.06 | Syncing Cyl

In [ ]:
import glob, os
from IPython.display import Image, display

print("Definition panels:")
for p in sorted(glob.glob('/content/designqa_synth/definition/*/*_sixpanel.png'))[:4]:
    print(os.path.basename(p)); display(Image(filename=p))

print("\nPresence (pos/neg) pairs:")
pos = sorted(glob.glob('/content/designqa_synth/presence/pos/*/*_closeup.png'))[:4]
neg = [p.replace('/pos/','/neg/') for p in pos]
for p, n in zip(pos, neg):
    print(os.path.basename(p))
    display(Image(filename=p))
    display(Image(filename=n))
